[View in Colaboratory](https://colab.research.google.com/github/hlecuanda/jupyter-notebooks-of-all-kinds/blob/master/DataExtractionFromPDFtoSheets.ipynb)

# Data extraction from PDF documents to Google Sheets


---

This is an example documenting how to extract data from a PDF document, in this case a Bank Statement from Chase Bank, taking advantage of Drive's export capability to capture text while doing document conversion (We instruct via API to "save" the PDF document as a Docs document, then we export the text from the Docs document using the 

    files().export(fileId=doc['id'],mimeType='text/plain')
    
API method.We will iterate thropugh a directory listing of such documents, extractig only the bits we areinterested in, reading each value (in this case, all the deposits ) into a python array. 

Finally we will iterate through this python array, creating cell objects from the `gsheets` library which gives us a nice pythonic way to tlk to the sheets API

Finally, we will send some requests to the raw Sheets API in order to autosize columns and furhter refine presentation on our worksheet. The reason to send raw requests to the API is because `gsheets` does not support v4 of the API yet where this methods were introduced. 

![spreadsheet][1]

The final product is a report that can be used to create pivot tables, more detailed analysis, perhaps export to *BigQuery*, or in this case, drive the automation of a terminal-only accounting package written in COBOL (Passport Software's PBS) having a terminal interface affords the possibility to import structured data via copy-paste by carefuly adding "keystroke" sequences to the datastream. This could be done directly in python, however a spreadsheet already exists wth the formulas necessary for that processing step, so we take advantage of that fact. 

[1]:https://docs.google.com/drawings/d/e/2PACX-1vRqdf_2TeKzERIkFVmtpxji2-2NmKATNVLh8nYXNmSN68peeSpHLvfpevHrwJsQr43OilHNPJpzmPik/pub?w=794&h=465

## Import auxiliary Libraries and software dependencies 

This section checks whether the required libraries have been installed in the current notebook. If this raises an exception, (For example, if the libraries are notavailable), the exception block will install the libraries.

The important auxiliary library is [gspread](http://gspread.readthedocs.io/en/latest/) plus the oauth2 client library and the GoogleAPI python client discovery based libraries.


In [0]:
try:
    from oauth2client.service_account import ServiceAccountCredentials
    from google.oauth2 import service_account
    import gspread
except:
    !pip install google-auth google-auth-httplib2 google-api-python-client gspread
    from oauth2client.service_account import ServiceAccountCredentials
    import gspread
    # from gspread.models import Cell
finally:
    import re
    import sys
    import httplib2
    from apiclient import discovery
    from google.colab import files
    from google.colab import output



## Authorization procedure

This is a modified Oauth2 procedure, used when [Domain Wide Delegation](https://developers.google.com/identity/protocols/OAuth2ServiceAccount?hl=en_US#delegatingauthority) has been applied. [The procedure has been taken from this StackOverflow](https://stackoverflow.com/questions/49374112/google-service-account-cant-impersonate-gsuite-user) reference. 

The authorization flow has been simplified using a service account instead of the usual auth flow. This requires:

*   A service account from the project's [API console's Credentials Section](https://console.cloud.google.com/apis/credentials?project=mezaops&organizationId=1063501829322). This will furnish a key file that you need to upload on this notebook
*   The client defined for the service account above, has to be authorized for the required scopes via the[ Domain's Admin Console](https://admin.google.com/lecuanda.com/AdminHome?chromeless=1#OGX:ManageOauthClients). 


The `try ... except` block will check whether the authorization file containing the encryption certificate has been loaded on the notebook. If it has not been loaded, it willopen a dialog for the user to upload the authorization file.

If you don't have the authorization file, please request it from your domain Admin. 

Finallym the `credentials.create_delegated()` method from the oauth2 clent will allow you to impersonate the user that has been used as parameter to this method.




In [0]:
SERVICE_ACCOUNT_FILE = 'MezaOps-9483d786f5ef.json'

SCOPES = ['https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.file',
          'https://www.googleapis.com/auth/drive.metadata.readonly',
          'https://www.googleapis.com/auth/spreadsheets']

try:
    with open(SERVICE_ACCOUNT_FILE, "r") as f:
        print()
except:
    uploaded = files.upload()

    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
                name=fn, length=len(uploaded[fn])))
finally:
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)


delegated_credentials = credentials.create_delegated('hector@lecuanda.com')  
delegated_http = delegated_credentials.authorize(httplib2.Http())
drive  = discovery.build('drive', 'v3', http=delegated_http)
sheets = discovery.build('sheets','v4', http=delegated_http)

## Report Parameters

Fill in this form with the Google drive URL Key of the file that contains the PDF statements to extract data from. It will update the `docs` variable to contain all documents to beprocessed

In [0]:
#@title Directory id { run: "auto", vertical-output: true, output-height: 100, form-width: "50%", display-mode: "form" }
dir_id = "1kghdjPxiHRvQY2sQa1koli2csSGqni8_" #@param {type:"string"}
drivequery="'{}' in parents and mimeType = \
'application/vnd.google-apps.document'".format(dir_id)
docs = drive.files().list(q=drivequery).execute()

print("{} documents selected".format(len(docs['files'])))


## Aux Routines

Here, we define two routines that do the actual data extraction using the drive API

In [0]:

def get_yymm(instring):
    ''' Gets the year and month for the statement being processed from the
        file name. We need this because the Chase Statements format omit 
        the year on the transaction detials'''
    yy=instring[0:4]
    mm=instring[4:6]
    return "{}-{}".format(yy,mm)

def get_text_from_statements(doclist):
    ''' Iterates through the drive files, exportsthe text, and filters it 
        through a compiled regular expresion, appending all data items to
        a list that will get returned. Also, correctthe date on each item by
        adding the year extracted fromthe filename'''
    deposits=[]
    ctr=0
    regex=re.compile('^\S{5}\sDeposit\s\S+$')
    with output.use_tags('extractdata'):
        sys.stdout.write('processing:\n')
        sys.stdout.flush()
    
    for doc in doclist:
        yymm=get_yymm(doc['name'])
        curr_statement=drive.files().export(fileId=doc['id'],mimeType='text/plain').execute()
        for line in curr_statement.decode().split('\r\n'):
            if regex.match(line):
                mo=line[0:2]
                dy=line[3:5]
                rest=line.split(' ')[2]
                deposits.append("{}-{},{:03.2f}".format(yymm,dy,float(rest.replace(",","").replace("$",""))))

                with output.use_tags('extractdata'):
                    sys.stdout.write('.')
                    sys.stdout.flush()
                ctr=ctr+1
                if ctr % 50 == 0:
                    with output.use_tags('extractdata'):
                        sys.stdout.write('{}\n'.format(ctr))
                        sys.stdout.flush()    
                    
    output.clear(output_tags='extractdata')
    print("processed {} records".format(ctr))
    return deposits

This call extracts the selected data, dumping it to the `data`  global var

In [0]:
data=get_text_from_statements(docs['files'])    

## Export data to a spreadhseet

### Helper functions for display on sheets

In [0]:
def pdate(isodate):
    '''
    >>> d='2015-02-01'
    >>> print(pdate(d))
    020115
    '''
    yy=isodate[2:4]
    mm=isodate[5:7]
    dd=isodate[8:]
    return "{}{}{}".format(mm,dd,yy)



def automation(datarow):
    '''
    >>> datarow = ['2015-02-01','1240.00']
    >>> print(automation(datarow))
    1010\n01\n020115\n020115\nDeposit from spreadsheet\n\n1099\n\01\n\e
    '''
    autostr='1010\\n01\\n{}\\n{}\\nDeposit from spreadsheet\\n\\n1099\\n\\01\\n\\e'
    return autostr.format(pdate(datarow[0]),pdate(datarow[0]),datarow[1])



This code block creates a Google Spreadsheet, selects a range and then dumps each value from the `data` variable into the newly created spreadsheet


In [0]:
gc = gspread.authorize(delegated_credentials)
Cell = gspread.models.Cell
shid = '15cCd61zbSO3Y4JrEolGg9WtT2w2Ig_KQRWGgrfe83os'

try:
    sh = gc.open_by_key(shid)
except:
    sh = gc.create('Deposits')
    shid = sh.id
finally:
    print(sh)
 
worksheet = sh.sheet1

if (worksheet.row_count < len(data)):
    extra_rows=len(data)-worksheet.row_count
    print("adding {} rows".format(extra_rows))
    worksheet.resize(len(data)+3,4)

print("new worksheet rows: {}".format(worksheet.row_count))

cells = []
for row in range(0,len(data)):
    datarow=[]
    
    yearcell=Cell(row+2,1 )
    monthcell=Cell(row+2,2 )
    yearmonthcell=Cell(row+2, 3)
    datecell=Cell(row+2,4)
    valuecell=Cell(row+2,5)
    formulacell=Cell(row+2,6)
    
    datarow=data[row].split(",")
    yearcell.value="=year(D{})".format(row+2)
    monthcell.value="=month(D{})".format(row+2)
    yearmonthcell.value='=join("-",A{},B{})'.format(row+2,row+2)
    datecell.value=datarow[0]
    valuecell.value=datarow[1]
    formulacell.value=automation(datarow)
    
    cells.append(yearcell)
    cells.append(monthcell)
    cells.append(yearmonthcell)
    cells.append(datecell)
    cells.append(valuecell)
    cells.append(formulacell)
    
worksheet.update_cells(cells,'USER_ENTERED')

worksheet.update_cell(1,1,"Year")
worksheet.update_cell(1,2,"Month")
worksheet.update_cell(1,3,"YearMonth")
worksheet.update_cell(1,4,"Date")
worksheet.update_cell(1,5,"Amount")
worksheet.update_cell(1,6,"Automation")

# Go to https://sheets.google.com to see your new spreadsheet.

### Spreadsheet formatting

#### Column Sorting

In [0]:
Sorting=json.dumps(''' 
      "sortRange": {
        "range": {
          "startRowIndex": 1,
          "endRowIndex": 1199,
          "startColumnIndex": 0,
          "endColumnIndex": 2,
          "sheetId": 0
        },
        "sortSpecs": [
          {
            "dimensionIndex": 0,
            "sortOrder": "ASCENDING"
          },
          {
            "dimensionIndex": 1,
            "sortOrder": "ASCENDING"
          }
        ]
      }
  ''')

#### Column Width Auto Adjustment

In [0]:
Autoadjust='''{
      "autoResizeDimensions": {
        "dimensions": {
          "sheetId": sheetId,
          "dimension": "COLUMNS",
          "startIndex": 0,
          "endIndex": 6
        }
      }
    }''' 

#### Formatting

In [0]:
CellFormat ='''{
      "repeatCell": {
        "range": {
          "sheetId": sheetId,
          "startRowIndex": 0,
          "endRowIndex": 10,
          "startColumnIndex": 0,
          "endColumnIndex": 6
        },
        "cell": {
          "userEnteredFormat": {
            "numberFormat": {
              "type": "NUMBER",
              "pattern": "#,##0.00"
            }
          }
        },
        "fields": "userEnteredFormat.numberFormat"
      }
    }'''


### Apply Formats

In [0]:
requests=[]
import json
import pprint

requests.append(Sorting)
requests.append(CellFormat)
requests.append(Autoadjust)
#requests.append()
body = {
    'requests': requests
}
response = sheets.spreadsheets().batchUpdate(spreadsheetId=shid,body=body).execute()
